In [3]:
import os
import numpy as np
import pandas as pd
import pickle
from scipy import stats 
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [4]:
src = "/home/andres/data/hbmep-processed/J_SHAP/data.csv"
df = pd.read_csv(src)

In [4]:
df.shape

(16440, 44)

In [5]:
columns = ["part"]

In [6]:
df.columns

Index(['pulse_amplitude', 'pulse_train_frequency', 'pulse_period',
       'pulse_duration', 'pulse_count', 'train_delay', 'channel1_1',
       'channel1_2', 'channel1_3', 'channel1_4', 'channel2_1', 'channel2_2',
       'channel2_3', 'channel2_4', 'charge_params_1', 'charge_params_2',
       'charge_params_3', 'charge_params_4', 'bank_check', 'run', 'enabled',
       'channel_failA', 'channel_failB', 'channel_fail_comb', 'time_pulse',
       'time', 'ix_onsets', 'auc_2', 'auc_6', 'auc_5', 'auc_7', 'auc_1',
       'auc_4', 'auc_3', 'auc_8', 'channel1_laterality', 'channel1_segment',
       'channel2_laterality', 'channel2_segment', 'compound_position',
       'compound_charge_params', 'participant', 'subdir_pattern',
       'charge_param_error'],
      dtype='object')

In [5]:
file = open("/home/andres/repos/rat-mapping-paper/reports/J_SHAP/combine.pkl","rb")
combo = pickle.load(file)

In [6]:
def threshold_array(shape):
    my_dict = {
    (key[0], key[1], key[3]): value["a"].mean() for  key,value in combo.items() if key[2] == shape
    }
    keys, values = zip(*my_dict.items())
    axis0 = np.unique([key[0] for key in keys])
    axis1 = ["Deltoid", "Biceps", "Triceps", "ECR", "FCR", "ADM"][::-1]
    pos_L = np.zeros((len(axis0), len(axis1)))
    pos_M = np.zeros((len(axis0), len(axis1)))
    for key, value in zip(keys, values):
        if key[1].split("-")[0][2:] == "L":
            pos_L[np.where(axis0 == key[0])[0], np.where(np.array(axis1) == key[2][1:])[0]] = 1/value 
        elif key[1].split("-")[0][2:] == "M":
            pos_M[np.where(axis0 == key[0])[0], np.where(np.array(axis1) == key[2][1:])[0]] = 1/value 
    return(pos_L, pos_M, axis0, axis1)

In [7]:
arr = None
responses = list(set([u[3] for u in combo.keys()]))
combinations_no_response = [(c[0], c[1], c[2]) for c in combo]
combinations_no_response = set(combinations_no_response)
combinations_no_response = list(combinations_no_response)
for muscle in responses:
    arr_muscle = None

    for c in combinations_no_response:
        a = combo[(*c, muscle)]["a"]
        a = a[:, 0, 0, 0, 0]

        if arr_muscle is None:
            arr_muscle = a[:, None]
        else:
            # concatenate now
            arr_muscle = np.concatenate([arr_muscle, a[:, None]], axis=-1)

    if arr is None:
        arr = arr_muscle[:,:, None]

    else:
        arr = np.concatenate([arr, arr_muscle[:,:, None]], axis = -1) 

In [10]:
arr.shape

(4000, 320, 6)

In [11]:
combinations_no_response

[('amap04', 'C7L-C6L', '50-0-50-0'),
 ('amap03', 'C7L-C7M', '80-0-20-400'),
 ('amap03', '-C7L', '20-0-80-25'),
 ('amap02', 'C8M-C8L', '50-0-50-100'),
 ('amap07', 'C6M-C6L', '50-0-50-100'),
 ('amap02', 'C7L-C8L', '50-0-50-0'),
 ('amap01', 'C8L-C7L', '20-0-80-25'),
 ('amap06', 'C7L-', '50-0-50-0'),
 ('amap08', '-C6L', '50-0-50-0'),
 ('amap02', 'C8L-C8M', '50-0-50-100'),
 ('amap03', 'C8L-', '80-0-20-400'),
 ('amap05', 'C7L-', '80-0-20-400'),
 ('amap02', 'C7L-C8L', '20-0-80-25'),
 ('amap08', '-C7L', '80-0-20-400'),
 ('amap06', 'C7L-', '20-0-80-25'),
 ('amap04', '-C7L', '50-0-50-100'),
 ('amap07', '-C6L', '80-0-20-400'),
 ('amap03', 'C7L-C7M', '50-0-50-0'),
 ('amap08', 'C7L-C7M', '80-0-20-400'),
 ('amap06', 'C6M-C6L', '80-0-20-400'),
 ('amap04', 'C6M-C6L', '20-0-80-25'),
 ('amap02', 'C7L-', '80-0-20-400'),
 ('amap03', 'C7L-C8L', '50-0-50-0'),
 ('amap05', 'C6L-C6M', '20-0-80-25'),
 ('amap05', 'C7L-', '50-0-50-0'),
 ('amap06', '-C7L', '50-0-50-0'),
 ('amap08', '-C7L', '50-0-50-0'),
 ('amap04'

In [8]:
df[['participant','compound_position', 'compound_charge_params']].apply(tuple, axis = 1).nunique()

320

In [9]:
pmono20 = np.array([bool(re.search(r'20-0-80-25', s[2])) for s in combinations_no_response])
pmono80 = np.array([bool(re.search(r'80-0-20-400', s[2])) for s in combinations_no_response])
mono = np.array([bool(re.search(r'50-0-50-100', s[2])) for s in combinations_no_response])
bip = np.array([bool(re.search(r'50-0-50-0', s[2])) for s in combinations_no_response])

lat = np.array([bool(s[1].split("-")[1][2:] == "L") for s in combinations_no_response])
med = np.array([bool(s[1].split("-")[1][2:] == "M") for s in combinations_no_response])



In [14]:
dictBool = {}

dictBool[('lateral pmono20')] = np.array([bool(re.search(r'20-0-80-25', s[2]) and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "L") for s in combinations_no_response])
dictBool[('lateral pmono80')] = np.array([bool(re.search(r'80-0-20-400', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "L") for s in combinations_no_response])
dictBool[('lateral bip')] = np.array([bool(re.search(r'50-0-50-100', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "L") for s in combinations_no_response])
dictBool[('lateral mono')] = np.array([bool(re.search(r'50-0-50-0', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "L") for s in combinations_no_response])

dictBool[('medial pmono20')] = np.array([bool(re.search(r'20-0-80-25', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "M") for s in combinations_no_response])
dictBool[('medial pmono80')] = np.array([bool(re.search(r'80-0-20-400', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "M") for s in combinations_no_response])
dictBool[('medial bip')] = np.array([bool(re.search(r'50-0-50-100', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "M") for s in combinations_no_response])
dictBool[('medial mono')] = np.array([bool(re.search(r'50-0-50-0', s[2])and re.search(r'.+-.+', s[1]) and s[1].split("-")[0][2:] == "M") for s in combinations_no_response])


In [15]:
value = []
for b,v in dictBool.items():
    for b1, v1 in dictBool.items():
        A = arr[:,v,:].mean(axis = 0).reshape(-1,)
        B = arr[:,v1,:].mean(axis = 0).reshape(-1,)
        tbvs, pbvs = stats.ttest_ind(B,A, alternative="less")
        if pbvs < 0.05:
            value.append(f"{b1} is more effective than {b} (p = {pbvs})")
        else: value.append(f"No significant difference for {b1} vs {b} (p = {pbvs})")

In [16]:
A = arr[:,mono,:].mean(axis = 0).reshape(-1,)
B = arr[:,bip,:].mean(axis = 0).reshape(-1,)

tbvs, pbvs = stats.ttest_ind(B,A, alternative="less")

pbvs

0.9999996519248618

In [17]:
value

['No significant difference for lateral pmono20 vs lateral pmono20 (p = 0.5)',
 'No significant difference for lateral pmono80 vs lateral pmono20 (p = 0.24885073758223752)',
 'No significant difference for lateral bip vs lateral pmono20 (p = 0.13371095690032497)',
 'No significant difference for lateral mono vs lateral pmono20 (p = 0.7217150520969153)',
 'No significant difference for medial pmono20 vs lateral pmono20 (p = 0.997775474786967)',
 'No significant difference for medial pmono80 vs lateral pmono20 (p = 0.9884524148383962)',
 'No significant difference for medial bip vs lateral pmono20 (p = 0.6939039894212642)',
 'No significant difference for medial mono vs lateral pmono20 (p = 0.9999413360281021)',
 'No significant difference for lateral pmono20 vs lateral pmono80 (p = 0.7511492624177625)',
 'No significant difference for lateral pmono80 vs lateral pmono80 (p = 0.5)',
 'No significant difference for lateral bip vs lateral pmono80 (p = 0.339606902913121)',
 'No significant d

In [18]:
a = set(u[1] for u in combinations_no_response)
a

{'-C6L',
 '-C7L',
 '-C8L',
 'C6L-',
 'C6L-C6M',
 'C6L-C7L',
 'C6M-C6L',
 'C7L-',
 'C7L-C6L',
 'C7L-C7M',
 'C7L-C8L',
 'C7M-C7L',
 'C8L-',
 'C8L-C7L',
 'C8L-C8M',
 'C8M-C8L'}

In [19]:
C61 = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C6" in s[1].split("-")[0][0:] or "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C71 = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C7" in s[1].split("-")[0][0:] or "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C81 = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C8" in s[1].split("-")[0][0:] or "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C62 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C6" in s[1].split("-")[0][0:] or "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C72 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C7" in s[1].split("-")[0][0:] or "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C82 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C8" in s[1].split("-")[0][0:] or "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C63 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C6" in s[1].split("-")[0][0:] or "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C73 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C7" in s[1].split("-")[0][0:] or "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C83 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C8" in s[1].split("-")[0][0:] or "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C64 = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C6" in s[1].split("-")[0][0:] or "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C74 = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C7" in s[1].split("-")[0][0:] or "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C84 = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C8" in s[1].split("-")[0][0:] or "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])

In [20]:
C = arr[:,C71,:].mean(axis = 0).reshape(-1,)
D = arr[:,C81,:].mean(axis = 0).reshape(-1,)

tbvs, pbvs = stats.ttest_ind(C,D, alternative="less")

pbvs

2.9022671961365798e-12

In [21]:
C74.sum()

48

In [22]:
arr[:,mask,:].shape

NameError: name 'mask' is not defined

In [ ]:
L, M = threshold_array("20-0-80-25")

In [ ]:
ttest_values, pscores = stats.ttest_ind(L, M)
pscores


array([0.92528306, 0.38606512, 0.67930961, 0.50053072, 0.51280737,
       0.35147554])

Graphing if we want group comparison

In [ ]:
def graph_arr(charge):
    L, M, subject, muscle = threshold_array(charge)
    sub_meanS = np.median(L, axis=0)
    rowS = np.vsplit(L,len(subject))
    rowS = [i.tolist() for i in rowS]
    sub_meanL = np.median(M, axis=0)
    rowL = np.vsplit(M,len(subject))
    rowL = [i.tolist() for i in rowL]
    for xval_ind, xval in enumerate(rowS):
        # plt.plot(xval[0], muscle, linestyle='dashed', marker=".", markersize=20, alpha =0.3, color = "pink")
        plt.plot(sub_meanS,muscle, color = "red", linestyle='dashed', marker=".", markersize=20, label="Lateral" if xval_ind == 0 else None)
        # plt.legend('Lateral')
    for xval_ind,xval in enumerate(rowL):
        # plt.plot(xval[0], muscle, linestyle='dashed', marker=".", markersize=25, alpha =0.3, color = "blue")
        plt.plot(sub_meanL,muscle, color = "black", linestyle='dashed', marker=".", markersize=20, label="Medial" if xval_ind == 1 else None)
        # plt.legend('Medial')
    plt.yticks(weight ='bold',style = 'italic')
    plt.xlabel('Efficacy')
    plt.legend()
    plt.show()
    

In [ ]:
graph_arr("20-0-80-25")

## Rostral Caudal

In [ ]:
a

{'-C6L',
 '-C7L',
 '-C8L',
 'C6L-',
 'C6L-C6M',
 'C6L-C7L',
 'C6M-C6L',
 'C7L-',
 'C7L-C6L',
 'C7L-C7M',
 'C7L-C8L',
 'C7M-C7L',
 'C8L-',
 'C8L-C7L',
 'C8L-C8M',
 'C8M-C8L'}

In [18]:
C67b = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C6" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C76b = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C78b = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C87b = np.array([bool(re.search(r'50-0-50-0', s[2]) and ("C8" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C67m = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C6" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C76m = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C78m = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C87m = np.array([bool(re.search(r'50-0-50-100', s[2]) and ("C8" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C67p80 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C6" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C76p80 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C78p80 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C87p80 = np.array([bool(re.search(r'80-0-20-400', s[2]) and ("C8" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C67p20 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C6" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C76p20 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C6" in s[1].split("-")[1][0:])) for s in combinations_no_response])

C78p20 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C7" in s[1].split("-")[0][0:] and "C8" in s[1].split("-")[1][0:])) for s in combinations_no_response])
C87p20 = np.array([bool(re.search(r'20-0-80-25', s[2]) and ("C8" in s[1].split("-")[0][0:] and "C7" in s[1].split("-")[1][0:])) for s in combinations_no_response])


In [20]:
rost = arr[:,C67m,:].mean(axis = 0).reshape(-1,)
caud = arr[:,C76m,:].mean(axis = 0).reshape(-1,)

stats.ttest_rel(rost,caud, alternative="less")

TtestResult(statistic=0.949306396515953, pvalue=0.8248452781115181, df=29)

In [ ]:
part = [s[1] for s in combinations_no_response if len(parts := s[1].split("-")) == 2 and parts[0] and parts[1] and parts[0][:2] != parts[1][:2]]
set(part)


{'C6L-C7L', 'C7L-C6L', 'C7L-C8L', 'C8L-C7L'}

## Ground 

In [ ]:
mybool = {}
mybool[('ground - position mono')] = np.array([bool(s[1].split("-")[0][2:] == "" and re.search(r'50-0-50-0', s[2])) for s in combinations_no_response])
mybool[('position - ground mono')] = np.array([bool(s[1].split("-")[1][2:] == "" and re.search(r'50-0-50-0', s[2])) for s in combinations_no_response])

mybool[('ground - position p20')] = np.array([bool(s[1].split("-")[0][2:] == "" and re.search(r'20-0-80-25', s[2])) for s in combinations_no_response])
mybool[('position - ground p20')] = np.array([bool(s[1].split("-")[1][2:] == "" and re.search(r'20-0-80-25', s[2])) for s in combinations_no_response])

mybool[('ground - position p80')] = np.array([bool(s[1].split("-")[0][2:] == "" and re.search(r'80-0-20-400', s[2])) for s in combinations_no_response])
mybool[('position - ground p80')] = np.array([bool(s[1].split("-")[1][2:] == "" and re.search(r'80-0-20-400', s[2])) for s in combinations_no_response])

mybool[('ground - position bip')] = np.array([bool(s[1].split("-")[0][2:] == "" and re.search(r'50-0-50-100', s[2])) for s in combinations_no_response])
mybool[('position - ground bip')] = np.array([bool(s[1].split("-")[1][2:] == "" and re.search(r'50-0-50-100', s[2])) for s in combinations_no_response])

In [ ]:
gvalue = []
for b,v in mybool.items():
    for b1, v1 in mybool.items():
        A = arr[:,v,:].mean(axis = 0).reshape(-1,)
        B = arr[:,v1,:].mean(axis = 0).reshape(-1,)
        tbvs, pbvs = stats.ttest_ind(B,A, alternative="less")
        if pbvs < 0.05:
            gvalue.append(f"{b1} is more effective than {b} (p = {pbvs})")
        else: gvalue.append(f"No significant difference for {b1} vs {b} (p = {pbvs})")

In [ ]:
gvalue

['No significant difference for ground - position mono vs ground - position mono (p = 0.5)',
 'No significant difference for position - ground mono vs ground - position mono (p = 0.9999999999999388)',
 'No significant difference for ground - position p20 vs ground - position mono (p = 0.5524870400308437)',
 'No significant difference for position - ground p20 vs ground - position mono (p = 0.9999999999972042)',
 'No significant difference for ground - position p80 vs ground - position mono (p = 0.9999996434502291)',
 'No significant difference for position - ground p80 vs ground - position mono (p = 0.42577787404621154)',
 'No significant difference for ground - position bip vs ground - position mono (p = 0.8182051477061763)',
 'No significant difference for position - ground bip vs ground - position mono (p = 0.5771767024003044)',
 'ground - position mono is more effective than position - ground mono (p = 6.115689694642182e-14)',
 'No significant difference for position - ground mono 